In [1]:
import findspark
findspark.init()
import pyspark
import random
import os

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
img_dir = "D:\\Data\\AnimalsOnTheWeb\\alligator"

In [4]:
imgs = spark.read.format("image").load(img_dir)

In [5]:
imgs.show()

+--------------------+
|               image|
+--------------------+
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
|[file:///D:/Data/...|
+--------------------+
only showing top 20 rows



In [6]:
imgs.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

